## Jupyter Notebook zur Berechnung der Levenshtein Distanz zwischen Maurer-Graphem und DWA-Graphem für jeden vorkommenden Abfrageort

Um die Levenshtein Distanzen pro Ort berechnen zu können, muss der Datensatz (Datensatz = jede csv Datei im "csv"-Ordner) zunächst nach Maurer- und DWA-Bögen getrennt werden.
Danach werden die Maurer- und DWA-Bögen so zusammengefasst, dass jeder Ort nur einmal im Datensatz vorkommt.
Die daraus resultierenden zwei Datensätze können dann anhand der Koordinaten zusammengeführt werden.
Das Ergebis ist ein Gesamtdatensatz, in dem für jeden Abfrageort alle Maurer- und DWA-Angaben gebündelt gegenübergestellt sind.
Dadurch kann pro Ort die Levenshtein Distanz (Minimale Anzahl an Substitutionen, Ergänzungen und Substraktionen, um einen String in einen zweiten zu überführen) zwischen Maurer- und DWA-Graphemen berechnet werden. Für Orte, an denen mehr als ein Maurer- bzw. DWA-Graphem vorliegt, wird die Distanz zwischen allen möglichen vorkommenden Kombinationen berechnet und der daraus resultierende Minimalwert in den Datensatz übernommen.
Orte, an denen kein Graphem vorliegt, bekommen die Kennziffer `999` zugeschrieben. Dies gilt auch für Orte, an denen die Grapheme nicht verglichen werden können, da kein Vergleichsdatensatz vorliegt.

In [1]:
import pandas as pd
from Levenshtein import distance
import os
import numpy as np

In [2]:
def make_dataframe(file):
    """
    Creates a pandas dataframe and drops "korrektur" and "org" columns
    
    Parameters:
    file(csv): must provide source to the csv file that is to be made into a dataframe
    
    Returns:
    df_sorted: pd.DataFrame
    """
    df = pd.read_csv(file, sep = "\t", index_col = False)
    df_sorted = df.drop(columns = ["korrektur", "org"])
    return df_sorted

In [3]:
def leven(val1, val2):
    """
    computes Levenshtein distance for two given values

    Parameters:
    val1(str or list): list must be comprised of str elements. Function checks if parameter is str or list. 
    If parameter is list, function will compute Levenshtein distance for each str within list.
    val2(str or list): list must be comprised of str elements. 
    Function checks if parameter is str or list. If parameter is list, function will compute Levenshtein distance for each str within list.

    Returns:
    minimum (int): minimal Levenshein distance computed for two given values
    """
    val1 = str(val1)
    val2 = str(val2)
    
    if ", " in val1:
        wert1 = val1.split(", ")
    else:
        wert1 = val1.strip()

    if ", " in val2:
        wert2 = val2.split(", ")
    else:
        wert2 = val2.strip()

    dist = []

    if type(wert1) == str and type(wert2) == str:
        if wert1 == "na" or wert1 == "nan" or wert2 == "na" or wert2 == "nan":
            dist_item = 999
        else:  
            dist_item = distance(wert1, wert2)
        dist.append(dist_item)

    elif type(wert1) == str and type(wert2) == list:
        for a in wert2:
            a = a.strip()
            if wert1 == "na" or wert1 == "nan" or a == "na" or a == "nan":
                dist_item = 999
            else:  
                dist_item = distance(wert1, a)
            dist.append(dist_item)

    elif type(wert2) == str and type(wert1) == list:
        for c in wert1:
            c = c.strip()
            if c == "na" or c == "nan" or wert2 == "na" or wert2 == "nan":
                dist_item = 999
            else:
                dist_item = distance(wert2, c)
            dist.append(dist_item)
    
    elif type(wert1) == list and type(wert2) == list:
        for l in wert1:
            l = l.strip()
            for b in wert2:
                b = b.strip()
                if l == "na" or l == "nan" or b == "na" or b == "nan":
                    dist_item = 999
                else:
                    dist_item = distance(l, b)
                dist.append(dist_item)
    
    minimum = min(dist)

    return minimum

In [4]:
def stringer(object):
    """
    changes datatype into str

    Parameters:
    object(instance of an "Ort" object)

    Returns:
    object
    """
    object.fragebogen = str(object.fragebogen)
    object.fragebogen2 = str(object.fragebogen2)
    object.ort = str(object.ort)
    object.item = str(object.item)
    object.item2 = str(object.item2)
    object.phontype = str(object.phontype)
    object.phontype2 = str(object.phontype2)
    object.lextype = str(object.lextype)
    object.lextype2 = str(object.lextype2)
    return object

In [5]:
def list_unique(befragung):
    """
    reverses questionnaire splitting and sums up data per location

    Parameters:
    befragung (list): takes in list of objects

    Returns:
    unique(list)
    """
    # In unique kommt jeder Ort und jeder Fragebogen nur 1 Mal vor
    unique = []

    # Belegsplittung rückgängig machen
    for object in befragung:
        if object.beleg == 1:
            unique.append(object)

    for object in befragung:
        if object.beleg > 1:
            for i in unique:
                if object.fragebogen == i.fragebogen:
                    i.add("na", object.ort, object.item, object.phontype, object.lextype)

    # Datensatz pro Ort zusammenfassen
    x = 0
    for i in unique:
        for j in unique[x::]:
            if i != j:
                if i.long == j.long and i.lat == j.lat:
                    i.add(j.fragebogen, j.ort, j.item, j.phontype, j.lextype)
                    unique.remove(j)
        x += 1
    
    return unique

In [6]:
class Ort:
    def __init__(self, fragebogen, ort, long, lat, item, phontype, lextype, beleg, erhebung):
        """
        Init function to create instance of "Ort" object.
        Takes in the information for each questionnaire given in the dataset.
        """
        self.fragebogen = fragebogen
        # fragebogen2, item2 etc. enthalten später die Grapheme aus dem DWA Bogen
        # fragebogen, item etc. enthalten die Grapheme aus der Maurerbefragung
        self.fragebogen2 = "na"
        self.ort = ort
        self.long = long
        self.lat = lat
        self.item = item
        self.item2 = "na"
        self.phontype = phontype
        self.phontype2 = "na"
        self.lextype = lextype
        self.lextype2 = "na"
        self.beleg = beleg
        self.erhebung = erhebung

    def add(self, fragebogen, ort, item, phontype, lextype):
        """
        function to add the name of the questionnaire, items, phontypes and lextypes to 
        an existing object instance in case there is more than one questionnaire per place

        Parameters:
        fragebogen(string)
        ort(string)
        item(sting)
        phontype(string)
        lextype(string)
    
        Returns:
        self: changed instance of self
        """
        self.fragebogen += f", {fragebogen}"
        self.ort += f", {ort}"
        self.item += f", {item}"
        self.phontype += f", {phontype}"
        self.lextype += f", {lextype}"

    def info(self):
        """
        provides information about an instance

        Parameters:
        self(instance of an object)

        Returns:
        print statement
        """
        print(self.fragebogen, self.fragebogen2, self.ort, self.item, self.item2, self.phontype, self.phontype2, self.lextype, self.lextype2)

In [7]:
path = "/home/kopatsch/Masterarbeit/MA/masterarbeit/data/csv/"
entries = os.listdir(path)

for entry in entries:
    df = make_dataframe(f"{path}{entry}")

    # Dataframe transponieren, um die nachfolgende Erstellung der Objectinstanzen mittels Verwendung von Indizes zu erleichtern
    df2 = df.T

    # Liste, die alle Objektinstanzen beinhaltet
    gesamt_liste = []
    for i in df2:
        # kreieren der Objektinstanzen
        ort_object = Ort(df2[i][1], df2[i][2], df2[i][4], df2[i][5], df2[i][6], df2[i][7], df2[i][8], df2[i][9], df2[i][10])
        gesamt_liste.append(ort_object)

    # Datensatz in einen Maurer- und einen DWA-Datensatz aufteilen
    maurer = []
    dwa = []
    for object in gesamt_liste:
        if object.erhebung == "maurer":
            maurer.append(object)
        elif object.erhebung == "DWA":
            dwa.append(object)

    # In maurer_unique kommt jeder Ort und jeder Fragebogen nur 1 Mal vor
    maurer_unique = list_unique(maurer)

    # In dwa_unique kommt jeder Ort und jeder Fragebogen nur 1 Mal vor
    dwa_unique = list_unique(dwa)

    # DWA- und Maurer-Datensätze mittels der Koordinaten zusammenführen
    final_list = []
    for i in maurer_unique:
        switch = True
        for j in dwa_unique:
            # da es möglich sein kann, dass mehr als 1 DWA Bogen zu einem Maurer Bogen gehört,
            # kann mit dem Switch überprüft werden, ob schon DWA Information im Object enthalten ist
            if i.lat == j.lat and i.long == j.long and switch == True:
                i.item2 = j.item
                i.phontype2 = j.phontype
                i.lextype2 = j.lextype
                i.fragebogen2 = j.fragebogen
                switch = False
            elif i.lat == j.lat and i.long == j.long and switch == False:
                i.item2 += ", " + j.item
                i.phontype2 += ", " + j.phontype
                i.lextype2 += ", " + j.lextype
                i.fragebogen2 += ", " + j.fragebogen
        final_list.append(i)

    for i in dwa_unique:
        # switch überprüft, ob DWA Daten bereits in der finalen Liste enthalten sind
        switch = True
        for j in final_list:
            if i.lat == j.lat and i.long == j.long:
                switch = False
        if switch == True:
            final_list.append(i)

    # Levenshtein Distanzen pro Ort berechnen
    lev_item_min= []
    lev_phontype_min = []
    lev_lextype_min = []

    for element in final_list:
        result1 = leven(element.item, element.item2)
        lev_item_min.append(result1)

        result2 = leven(element.phontype, element.phontype2)
        lev_phontype_min.append(result2)

        result3 = leven(element.lextype, element.lextype2)
        lev_lextype_min.append(result3)

    # Daten nach Kategorien sortiert in Listen abspeichern
    # Listen zu einem pandas DataFrame zusammenführen
    df3 = pd.DataFrame()

    fragebogen_maurer = []
    fragebogen_dwa = []
    ort = []
    long = []
    lat = []
    item_maurer = []
    item_dwa = []
    phontype_maurer = []
    phontype_dwa = []
    lextype_maurer = []
    lextype_dwa = []

    for i in final_list:
        i.fragebogen = i.fragebogen.replace(", na", "")
        i.fragebogen2 = i.fragebogen2.replace(", na", "")
        if i.erhebung == "maurer":
            stringer(i)
            fragebogen_maurer.append(i.fragebogen)
            fragebogen_dwa.append(i.fragebogen2)
            ort.append(i.ort)
            long.append(i.long)
            lat.append(i.lat)
            item_maurer.append(i.item)
            item_dwa.append(i.item2)
            phontype_maurer.append(i.phontype)
            phontype_dwa.append(i.phontype2)
            lextype_maurer.append(i.lextype)
            lextype_dwa.append(i.lextype2)
        elif i.erhebung == "DWA":
            stringer(i)
            fragebogen_maurer.append(i.fragebogen2)
            fragebogen_dwa.append(i.fragebogen)
            ort.append(i.ort)
            long.append(i.long)
            lat.append(i.lat)
            item_maurer.append(i.item2)
            item_dwa.append(i.item)
            phontype_maurer.append(i.phontype2)
            phontype_dwa.append(i.phontype)
            lextype_maurer.append(i.lextype2)
            lextype_dwa.append(i.lextype)

    df3["Fragebogen-Nr. Maurer"] = fragebogen_maurer
    df3["Fragebogen-Nr. DWA"] = fragebogen_dwa
    df3["Ort"] = ort
    df3["Longitude"] = long
    df3["Latitude"] = lat
    df3["Item Maurer"] = item_maurer
    df3["Item DWA"] = item_dwa
    df3["Levenshtein Item Min"] = lev_item_min
    df3["Phontype Maurer"] = phontype_maurer
    df3["Phontype DWA"] = phontype_dwa
    df3["Levenshtein Phontype Min"] = lev_phontype_min
    df3["Lextype Maurer"] = lextype_maurer
    df3["Lextype DWA"] = lextype_dwa
    df3["Levenshtein Lextype Min"] = lev_lextype_min

    filename = entry.split("Maurer_")
    filename = filename[1].lower()
    # speichern
    df3.to_csv(f"/home/kopatsch/Masterarbeit/MA/masterarbeit/data/levenshtein_single_files/{filename}", sep = "\t")

In [8]:
# Liste der Untersuchungsworte
# Vereinfacht die Automatisierungen verschidener Funktionen im weiteren Scriptverlauf
words = ["ameise", "begräbnis", "deichsel", "elster", "fledermaus", "gurke", "hagebutte", "hebamme", "kartoffel", "maulwurf", "pflaume", "stecknadel", "ziege", "zimmerfliege"]

levenshtein_item = []
levenshtein_phon= []
levenshtein_lex = []
for word in words:
    df = pd.read_csv(f"/home/kopatsch/Masterarbeit/MA/masterarbeit/data/levenshtein_single_files/{word}.csv", sep= "\t")
    df1 = df.drop(df[df["Levenshtein Item Min"] == 999].index)
    liste1 = list(df1["Levenshtein Item Min"])
    for x in liste1:
        levenshtein_item.append(x)
        levenshtein_phon.append(x)
        levenshtein_lex.append(x)
    df2 = df.drop(df[df["Levenshtein Phontype Min"] == 999].index)
    liste2 = list(df2["Levenshtein Phontype Min"])
    for x in liste2:
        levenshtein_phon.append(x)
        levenshtein_lex.append(x)
    df3 = df.drop(df[df["Levenshtein Lextype Min"] == 999].index)
    liste3 = list(df3["Levenshtein Lextype Min"])
    for x in liste3:
        levenshtein_lex.append(x)


mittel_item = np.mean(levenshtein_item)
stvar_item = np.std(levenshtein_item)
mittel_phon = np.mean(levenshtein_phon)
stvar_phon = np.std(levenshtein_phon)
mittel_lex = np.mean(levenshtein_lex)
stvar_lex = np.std(levenshtein_lex)


for word in words:
    df = pd.read_csv(f"/home/kopatsch/Masterarbeit/MA/masterarbeit/data/levenshtein_single_files/{word}.csv", sep= "\t")
    df.insert(loc = 9, column = "z_score_item", value = ((df["Levenshtein Item Min"]-mittel_item)/stvar_item))
    df.insert(loc = 13, column = "z_score_phontype", value = ((df["Levenshtein Phontype Min"]-mittel_phon)/stvar_phon))
    df.insert(loc = 17, column = "z_score_lextype", value = ((df["Levenshtein Lextype Min"]-mittel_lex)/stvar_lex))
    df = df.drop(columns = ["Unnamed: 0"])
    df.to_csv(f"/home/kopatsch/Masterarbeit/MA/masterarbeit/data/levenshtein_single_files/{word}.csv", sep= "\t")